# **SETUP**

## Import

In [47]:
import numpy as np
import pickle
import os
from __future__ import division
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

seed = 76424236
np.random.seed(seed=seed)

folder_name = f"./Dataset/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

# **FUNCTION DEFINITION**

In [48]:
def calc_metrics(infilename, trainfilename, gamma=0.2, K=10):
    
    infile = open(infilename, 'rb')
    P = pickle.load(infile)
    infile.close()
    NUM_NEGATIVES = P["num_negatives"]
    _NUM_POSs = dict()
    
    for theuser in P["users"]:
        _NUM_POSs[theuser] = len(P["user_items"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(P["user_items"][theuser])[-300:]
        P["results"][theuser] = list(P["results"][theuser])[-300:]
    
    Zui = dict()
    Ni = dict()
    
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][0 - _NUM_POSs[theuser]:]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
                
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][0 - _NUM_POSs[theuser]:]
        pos_scores = P["results"][theuser][0 - _NUM_POSs[theuser]:]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
            
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][0 - _NUM_POSs[theuser]:]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator
    
    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }


# **VALIDATE MODEL**

## Load training set

In [50]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['val_data'] = np.load(folder_name + "validation_arr.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
val_dataset = ImplicitDataset(raw_data['val_data'], raw_data['max_user'], raw_data['max_item'], name='Val')

## Define model

In [51]:
# Avoid tensorflow using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

## Validate

In [53]:
cml_model.load(folder_name+"cml-yahoo")

model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 300
model_trainer._exclude_positives([train_dataset, val_dataset])
model_trainer._sample_negatives(seed=10)
model_trainer._eval_save_prefix = folder_name+"cml-yahoo-val-new"
model_trainer._evaluate_partial(val_dataset)

[Subsampling negative items]


100%|██████████| 7396/7396 [00:03<00:00, 1930.89it/s]  


{'AUC': [0.8505555555555554,
  0.90375,
  0.9933333333333334,
  0.9716666666666667,
  0.9933333333333333,
  0.8183333333333334,
  0.9516666666666667,
  0.9171428571428571,
  0.9716666666666667,
  0.7417307692307694,
  0.9555555555555556,
  0.7466666666666667,
  0.5816666666666667,
  0.7466666666666666,
  0.7933333333333333,
  0.9750000000000001,
  0.9266666666666666,
  0.9141666666666667,
  0.8944444444444444,
  1.0,
  0.9966666666666667,
  0.9583333333333334,
  0.8233333333333334,
  0.6826666666666668,
  0.9916666666666667,
  0.9666666666666667,
  1.0,
  0.59,
  0.7161904761904762,
  0.935,
  0.9158333333333333,
  0.9166666666666666,
  0.9138888888888891,
  0.8926666666666666,
  0.9533333333333334,
  0.9583333333333333,
  0.9966666666666667,
  0.9766666666666667,
  0.8283333333333334,
  1.0,
  0.65,
  0.99,
  0.98,
  0.5633333333333334,
  0.9433333333333334,
  0.9933333333333333,
  0.7366666666666667,
  0.7333333333333334,
  0.8608333333333333,
  0.9933333333333333,
  0.90833333333333